In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
# Imports
import tensorflow as tf
import numpy as np
from arc_code.data import get_data
import math
import matplotlib.pyplot as plt

In [2]:
# getting the data
grouped_X_train, _, grouped_X_test, _ = get_data('../data', training_set = True, grouped_by_problem = True)

In [3]:
def get_largest_grid(X_train):
    # get largest grid
    max_width, max_height = -1, -1
    for x_arr in X_train:
        for x in x_arr:
            if x.shape[0] > max_height:
                max_height = x.shape[0]
            if x.shape[1] > max_width:
                max_width = x.shape[1]
    return np.array([max_height, max_width])

In [81]:
def pad(grid, shape_to_pad_to, fill = -1):
    # pads the grid to be of shape <shape_to_pad_to> with
    # fill values <fill>
    x_resize, y_resize = (shape_to_pad_to -grid.shape )/2.0
    
    def transform_format(arr):
        if arr%1 != 0:
            arr = [math.floor(arr),math.ceil(arr)]
        else:
            arr = [int(arr), int(arr)]
        return arr
    
    x_resize, y_resize = transform_format(x_resize), transform_format(y_resize)
    padded = np.pad(grid,[x_resize,y_resize,], 'constant', constant_values=fill)
    # add last layer and convert to float 32
    return padded[:,:,np.newaxis].astype(np.float32)

In [82]:
def get_cluster_data(data, max_shape):
    # key = grid, value = which problem it belongs to
    X, Y = [], []
    for i,x_arrs in enumerate(data):
        for x in x_arrs:
            X.append(pad(x, max_shape, fill = -1))
            Y.append(i)
            
    X = np.array(X)
    Y = np.array(Y)
    return X,Y

In [83]:
max_shape = get_largest_grid(grouped_X_train)
X_train, Y_train = get_cluster_data(grouped_X_train, max_shape)
X_test, Y_test = get_cluster_data(grouped_X_test, max_shape)

In [93]:
def one_layer_flat_model(num_classes, flattened_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=np.append(flattened_shape,1)),
        #keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(num_classes)
    ])
    return model

In [107]:
def one_layer_conv_model(num_classes, input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(
            3,
            3,
            strides=(1, 1),
            padding="valid",
            activation='relu',
            kernel_regularizer = tf.keras.regularizers.L1L2(),
            activity_regularizer = tf.keras.regularizers.L1L2(),
            bias_regularizer = tf.keras.regularizers.L1L2(),
            input_shape=np.append(input_shape,1)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes)
    ])
    return model

In [105]:
def two_layer_conv_model(num_classes, input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(
            3,
            3,
            strides=(1, 1),
            padding="valid",
            activation='relu',
            kernel_regularizer = tf.keras.regularizers.L1L2(),
            activity_regularizer = tf.keras.regularizers.L1L2(),
            bias_regularizer = tf.keras.regularizers.L1L2(),
            input_shape=np.append(input_shape,1)),
        tf.keras.layers.Conv2D(
            3,
            3,
            strides=(1, 1),
            padding="valid",
            kernel_regularizer = tf.keras.regularizers.L1L2(),
            activity_regularizer = tf.keras.regularizers.L1L2(),
            bias_regularizer = tf.keras.regularizers.L1L2(),
            activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes)
    ])
    return model

In [99]:
def cluster(model_func, X_train, Y_train, X_test, Y_test, num_epochs, *args):
    model = model_func(*args)
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    model.fit(X_train, Y_train, epochs=num_epochs, validation_data = (X_test, Y_test))
    test_loss, test_acc = model.evaluate(X_test,  Y_test, verbose=2)
    return model

In [98]:
cluster(one_layer_flat_model,X_train,Y_train,X_test, Y_test, 70,
                len(grouped_X_train), max_shape)

Train on 1301 samples, validate on 416 samples
Epoch 1/70
1301/1301 [==============================] - 1s 870us/sample - loss: 21.7562 - accuracy: 0.0000e+00 - val_loss: 5.5785 - val_accuracy: 0.0000e+00
Epoch 2/70
1301/1301 [==============================] - 0s 303us/sample - loss: 6.2965 - accuracy: 0.0000e+00 - val_loss: 4.6484 - val_accuracy: 0.0000e+00
Epoch 3/70
1301/1301 [==============================] - 0s 305us/sample - loss: 3.8511 - accuracy: 0.0000e+00 - val_loss: 4.9060 - val_accuracy: 0.0000e+00
Epoch 4/70
1301/1301 [==============================] - 0s 269us/sample - loss: 1.8947 - accuracy: 0.0000e+00 - val_loss: 3.8303 - val_accuracy: 0.0000e+00
Epoch 5/70
1301/1301 [==============================] - 0s 276us/sample - loss: 1.6860 - accuracy: 0.0000e+00 - val_loss: 4.2798 - val_accuracy: 0.0000e+00
Epoch 6/70
1301/1301 [==============================] - 0s 357us/sample - loss: 0.8190 - accuracy: 0.0000e+00 - val_loss: 3.8460 - val_accuracy: 0.0000e+00
Epoch 7/70
1301/

Epoch 53/70
1301/1301 [==============================] - 0s 187us/sample - loss: 0.0074 - accuracy: 0.0000e+00 - val_loss: 3.6545 - val_accuracy: 0.0000e+00
Epoch 54/70
1301/1301 [==============================] - 0s 257us/sample - loss: 0.0028 - accuracy: 0.0000e+00 - val_loss: 3.6523 - val_accuracy: 0.0000e+00
Epoch 55/70
1301/1301 [==============================] - 0s 232us/sample - loss: 0.0065 - accuracy: 0.0000e+00 - val_loss: 3.6607 - val_accuracy: 0.0000e+00
Epoch 56/70
1301/1301 [==============================] - 0s 214us/sample - loss: 0.0030 - accuracy: 0.0000e+00 - val_loss: 3.6605 - val_accuracy: 0.0000e+00
Epoch 57/70
1301/1301 [==============================] - 0s 191us/sample - loss: 0.0023 - accuracy: 0.0000e+00 - val_loss: 3.6589 - val_accuracy: 0.0000e+00
Epoch 58/70
1301/1301 [==============================] - 0s 374us/sample - loss: 0.0023 - accuracy: 0.0000e+00 - val_loss: 3.6589 - val_accuracy: 0.0000e+00
Epoch 59/70
1301/1301 [==============================] - 1

In [108]:
cluster(one_layer_conv_model,X_train,Y_train,X_test, Y_test, 70,
                len(grouped_X_train), max_shape)

Train on 1301 samples, validate on 416 samples
Epoch 1/70
1301/1301 [==============================] - 3s 2ms/sample - loss: 6.4720 - accuracy: 0.0200 - val_loss: 5.4469 - val_accuracy: 0.0601
Epoch 2/70
1301/1301 [==============================] - 3s 2ms/sample - loss: 3.8841 - accuracy: 0.3636 - val_loss: 5.0806 - val_accuracy: 0.1274
Epoch 3/70
1301/1301 [==============================] - 3s 2ms/sample - loss: 2.8300 - accuracy: 0.5926 - val_loss: 5.1671 - val_accuracy: 0.1683
Epoch 4/70
1301/1301 [==============================] - 3s 2ms/sample - loss: 2.2276 - accuracy: 0.7033 - val_loss: 5.3538 - val_accuracy: 0.1779
Epoch 5/70
1301/1301 [==============================] - 2s 2ms/sample - loss: 1.7988 - accuracy: 0.7586 - val_loss: 5.6826 - val_accuracy: 0.2019
Epoch 6/70
1301/1301 [==============================] - 2s 1ms/sample - loss: 1.4982 - accuracy: 0.8032 - val_loss: 6.1090 - val_accuracy: 0.2115
Epoch 7/70
1301/1301 [==============================] - 2s 2ms/sample - loss:

In [106]:
cluster(two_layer_conv_model,X_train,Y_train,X_test, Y_test, 70,
                len(grouped_X_train), max_shape)

Train on 1301 samples, validate on 416 samples
Epoch 1/70
1301/1301 [==============================] - 4s 3ms/sample - loss: 5.9559 - accuracy: 0.0123 - val_loss: 5.7137 - val_accuracy: 0.0457
Epoch 2/70
1301/1301 [==============================] - 3s 2ms/sample - loss: 4.6396 - accuracy: 0.3405 - val_loss: 5.1009 - val_accuracy: 0.1154
Epoch 3/70
1301/1301 [==============================] - 3s 2ms/sample - loss: 2.8182 - accuracy: 0.6018 - val_loss: 5.4685 - val_accuracy: 0.1899
Epoch 4/70
1301/1301 [==============================] - 2s 2ms/sample - loss: 1.7622 - accuracy: 0.7164 - val_loss: 5.8647 - val_accuracy: 0.2067
Epoch 5/70
1301/1301 [==============================] - 3s 2ms/sample - loss: 1.1610 - accuracy: 0.8017 - val_loss: 6.5315 - val_accuracy: 0.2284
Epoch 6/70
1301/1301 [==============================] - 3s 2ms/sample - loss: 0.8578 - accuracy: 0.8424 - val_loss: 7.5797 - val_accuracy: 0.2212
Epoch 7/70
1301/1301 [==============================] - 3s 2ms/sample - loss:

KeyboardInterrupt: 